In [22]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h", sampling_rate=44100)
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [23]:
import pandas as pd
column_names = [f'feature_{i+1}' for i in range(32)]
column_names += ['track_id', 'label']
df = pd.DataFrame(columns=column_names)

In [24]:
import torchaudio
import torch
import os

# Supondo que df já foi definido anteriormente
# df = pd.DataFrame(columns=['Col1', 'Col2', ..., 'TrackID'])

directory_path = "../../audios/fma_small"
arquivos_problematicos = []  # Lista para guardar os nomes dos arquivos problemáticos

i = 0
for root, dirs, files in os.walk(directory_path):
    if i == 5:
        break
    for filename in files:
        if filename.endswith(".mp3"):
          
            try:
                if i == 5:
                    break
                file_path = os.path.join(root, filename)
                track_id = filename.split('.')[0].lstrip("0")
                print(f"Processando {file_path}...")
                print(f"Track ID: {track_id}")
                print(f"Progresso: {i} arquivos processados")
                
                audio_input, rate = torchaudio.load(file_path)
                print(rate)
                input_values = processor(audio_input.squeeze().numpy(), return_tensors="pt", padding="longest").input_values
                # Extract features by passing the input through the model
                with torch.no_grad():
                    features = model(input_values).logits
                # Aggregate features using mean pooling
                aggregated_features = torch.mean(features, dim=1).squeeze().numpy()
                new_row_data = list(aggregated_features) + [track_id]
                df.loc[len(df)] = new_row_data
                i+= 1
            except Exception as e:  # Captura qualquer exceção que ocorra
                print(f"Erro ao processar {filename}: {e}")
                arquivos_problematicos.append(filename)  # Adiciona o nome do arquivo à lista

# Aqui, você pode lidar com os arquivos_problematicos como desejar
print("Arquivos problemáticos:", arquivos_problematicos)


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Processando ../../audios/fma_small/135/135054.mp3...
Track ID: 135054
Progresso: 0 arquivos processados
44100


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Erro ao processar 135054.mp3: cannot set a row with mismatched columns
Processando ../../audios/fma_small/135/135336.mp3...
Track ID: 135336
Progresso: 0 arquivos processados
44100


KeyboardInterrupt: 

In [ ]:
df.to_csv('wav2vec-fma-small.csv', index=False)

In [ ]:
with open('wav2vec-fma-small-failed.txt', 'w') as file:
    # Write each item on a new line
    for item in arquivos_problematicos:
        file.write(f"{item}\n")

In [ ]:
df = pd.read_csv('wav2vec-fma-small.csv')